In [1]:
# # for logging 

from comet_ml import Experiment
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.loggers import TestTubeLogger
from pytorch_lightning.loggers import WandbLogger
import wandb

# For metrics
from pytorch_lightning import metrics

import math
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import io
import torchtext
from torchtext.utils import download_from_url, extract_archive
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import pytorch_lightning as pl
from pytorch_lightning.trainer.trainer import Trainer





from pytorch_lightning.callbacks.early_stopping import EarlyStopping # The EarlyStopping callback can be used to monitor a validation metric and stop the training when no improvement is observed.
"""
    To enable it:

    Import EarlyStopping callback.

    Log the metric you want to monitor using log() method.

    Init the callback, and set monitor to the logged metric of your choice.

    Pass the EarlyStopping callback to the Trainer callbacks flag.
"""

from pytorch_lightning import seed_everything
seed_everything(42)

42

In [2]:
from sklearn.model_selection import train_test_split

class AlarmDataset(Dataset):
    def __init__(self,data,seq_len,batch_size):
        self.length = len(data)//seq_len # how much data i have         
        self.data = data
        self.seq_len = seq_len
        self.batch_size = batch_size
       
    def __getitem__(self, index: int):
        x = self.data[index*self.seq_len:(index*self.seq_len)+self.seq_len]
        y = self.data[1+index*self.seq_len:1+(index*self.seq_len)+self.seq_len]
        return x,y
    
    def __len__(self) -> int:
        return self.length

class MyDataModule(pl.LightningDataModule):
    
    def __init__(self, dir_path:str, file_name:str, config):
        super().__init__()
        # self.batch_size = batch_size
        # self.data_path = data_path
        self.config = config

        self.tokenizer = get_tokenizer('basic_english')
        self.vocab = build_vocab_from_iterator(map(self.tokenizer,iter(io.open(dir_path+file_name,encoding="utf8"))))
                
        # url = data_path
        # test_filepath, valid_filepath, train_filepath = extract_archive(download_from_url(url))
        seqs = None
        with open(dir_path+file_name) as f:
            seqs = f.readlines()
        seqs = [seq for seq in seqs if len(seq.split())<=self.config['filter-seq-len']]

        print(f"total seqs= {len(seqs)}")
        print(seqs[:4])
        train, valid = train_test_split(seqs,test_size=0.30,shuffle=False)
        valid, test = train_test_split(valid,test_size=0.30, shuffle=False)

        with open(dir_path +"train.tokens","w") as f:
            for seq in train:
                f.write(seq)
        
        with open(dir_path +"val.tokens","w") as f:
            for seq in valid:
                f.write(seq)
            
        with open(dir_path +"test.tokens","w") as f:
            for seq in test:
                f.write(seq)

        train_data = self.data_process(iter(io.open(dir_path +"train.tokens", encoding="utf8")))
        val_data = self.data_process(iter(io.open(dir_path +"val.tokens", encoding="utf8")))
        test_data = self.data_process(iter(io.open(dir_path +"test.tokens", encoding="utf8")))

    
        self.train_dataset = AlarmDataset(train_data, self.config['seq-len'], self.config['batch-size'])
        self.valid_dataset = AlarmDataset(val_data,self.config['seq-len'], self.config['batch-size'])
        self.test_dataset = AlarmDataset(test_data, self.config['seq-len'], self.config['batch-size'])

    
    def data_process(self, raw_text_iter):
        data = [torch.tensor([self.vocab[token] for token in self.tokenizer(item)],dtype=torch.long) for item in raw_text_iter]
        return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))


    def prepare_data(self):
        """
            Use this method to do things that might write to disk or that need to be done only from a single GPU in distributed settings.
            e.g., download,tokenize,etc…
        """ 
        return None


    def setup(self, stage: None):
        """
            There are also data operations you might want to perform on every GPU. Use setup to do things like:
            count number of classes,build vocabulary,perform train/val/test splits,apply transforms (defined explicitly in your datamodule or assigned in init),etc…
        """
        return None

    def train_dataloader(self) -> DataLoader:
        return DataLoader(self.train_dataset, batch_size=self.config['batch-size'], shuffle=False,num_workers=8,drop_last=True, pin_memory=True)
    
    def val_dataloader(self) -> DataLoader:
        return DataLoader(self.valid_dataset, batch_size=self.config['batch-size'], shuffle=False,num_workers=8,drop_last=True, pin_memory=True)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.config['batch-size'], shuffle=False,num_workers=8,drop_last=True, pin_memory=True)



In [3]:
class PositionalEncoding(torch.nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [4]:
class TransformerModel(pl.LightningModule):

    def __init__(self, config):
        super(TransformerModel, self).__init__()
        self.config = config        
        self.lr = self.config["lr"]
        self.weight_decay = self.config["weight-decay"]
    
        self.pos_encoder = PositionalEncoding(self.config['em-size'], self.config['dropout'])
        encoder_layers = torch.nn.TransformerEncoderLayer(self.config['em-size'], self.config['nhead'], self.config['nhid'], self.config["dropout"])
        self.transformer_encoder = torch.nn.TransformerEncoder(encoder_layers, self.config['nlayers'])
        self.encoder = torch.nn.Embedding(self.config["vocab-size"], self.config['em-size'])
        self.decoder = torch.nn.Linear(self.config['em-size'], self.config["vocab-size"])
        self.src_mask = self.generate_square_subsequent_mask(self.config['seq-len'])
        self.init_weights()

        self.train_F1 = metrics.classification.F1(num_classes=self.config["vocab-size"])
        self.val_F1 = metrics.classification.F1(num_classes=self.config["vocab-size"])
        self.test_F1 = metrics.classification.F1(num_classes=self.config["vocab-size"])


        self.log("Sequence length",self.config['seq-len'])
        self.log("lr",self.lr)
        self.log("# of tokens/vocab_size (unique alarms)",self.config['vocab-size'])
        self.log("weight_decay",self.weight_decay)
        self.save_hyperparameters()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src_mask = src_mask.to(self.device)
        src = self.encoder(src) * math.sqrt(self.config['em-size'])
        src = self.pos_encoder(src)
        src_mask = src_mask.to(self.device)
      
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        
        return output

   # The ReduceLROnPlateau scheduler requires a monitor
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr,weight_decay=self.weight_decay)
        d = {
       'optimizer': optimizer,
       'lr_scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = "min", factor = 0.5, patience=4, verbose=True),
       'monitor': 'val_epoch_loss',
        'interval': 'epoch'
        }
        return d    
    
    def training_step(self,batch,batch_idx):
        x,y = batch
        x = x.T
        y = y.T.reshape(-1)

        if x.size(0) != self.config['seq-len']:
           self.src_mask =  self.generate_square_subsequent_mask(x.size(0))
        
        y_hat = self(x,self.src_mask)
        y_hat =  y_hat.view(-1, self.config['vocab-size'])
        loss = F.cross_entropy(y_hat,y) # cross entropy itself compute softmax 
        
        self.log('train_loss',loss,logger=True)
        self.log('train_F1',self.train_F1(F.softmax(y_hat),y),logger=True)
        return loss

    def validation_step(self,batch, batch_idx):
        x,y = batch
        x = x.T
        y = y.T.reshape(-1)
        
        if x.size(0) != self.config['seq-len']:
           self.src_mask =  self.generate_square_subsequent_mask(x.size(0))
        
        y_hat = self(x,self.src_mask)
        y_hat =  y_hat.view(-1, self.config['vocab-size'])
        loss = F.cross_entropy(y_hat,y)

        self.log('val_loss',loss,logger=True)
        self.log('val_F1',self.val_F1(F.softmax(y_hat) ,y),logger=True)
        return {'val_loss':loss}
    
    def test_step(self,batch, batch_idx):
        x,y = batch
        x = x.T
        y = y.T.reshape(-1)
        if x.size(0) != self.config['seq-len']:
           self.src_mask =  self.generate_square_subsequent_mask(x.size(0))

        y_hat = self(x,self.src_mask)
        y_hat =  y_hat.view(-1,  self.config['vocab-size'])
        loss = F.cross_entropy(y_hat,y)

        self.test_F1(F.softmax(y_hat) ,y)
        
        self.log('test_loss',loss,logger=True)
        self.log('test_F1', self.test_F1(F.softmax(y_hat) ,y),logger=True)
        return {'test_loss':loss}
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([d['loss']  for d in outputs]).mean()
        f1 = self.train_F1.compute()
        print(f">Epoch ={self.current_epoch}, Avg Training loss = {avg_loss}, F1 = {f1}")
        self.log("train_epoch_loss",avg_loss,logger=True,prog_bar=True)
        self.log("train_epoch_F1", f1, logger=True,prog_bar=True)
  
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([d['val_loss'] for d in outputs]).mean()
        f1 = self.val_F1.compute()
        print(f">== Average Valid Loss = {avg_loss}, F1 = {f1}")
        self.log("val_epoch_loss",avg_loss,logger=True)
        self.log("val_epoch_F1",f1,logger=True,prog_bar=True)
    
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([d['test_loss'] for d in outputs]).mean()
        f1 = self.test_F1.compute()
        print(f">Average Test Loss = {avg_loss}, f1= {f1}")
        self.log("test_epoch_loss",avg_loss, logger = True)
        self.log("test_epoch_F1",f1, logger=True)
        self.save_hyperparameters()
    

# Trainning

**Note: When monitoring any parameter after the validation epoch end then you should pass check_val_every_n_epoch=1  not to other. This is very important.**

## Loggers

In [5]:
wandb_logger = WandbLogger(project="Alarm-Transformers-Net")
# comet_logger = CometLogger(
#     api_key="YZWScOiWdE8FwQSUj725dRmor",
#     project_name="Alarm-Transformers-Net" # Optional
# )
# test_tube_logger = TestTubeLogger('tb_logs', name='Alarm-Transformers-Net')

# Hyperparameters Configuration
## Hyperparameter optimization with W&B sweeps

So far, we chose our learning rate and number of layers arbitrarily. We can automate hyperparameter search with [W&B sweeps](https://docs.wandb.com/sweeps).

## Defining sweep configuration

Sweeps can be defined in multiple ways:
* with a YAML file - best for distributed sweeps and runs from command line
* with a Python object - best for notebooks


The main items to be defined in a sweep are:
*   **Metric** – This is the metric the sweeps are attempting to optimize. Metrics can take a `name` (this metric should be logged by your training script) and a `goal` (maximize or minimize). 
*   **Search Strategy** – We support several different search strategies with sweeps – `grid`, `random`, `bayes`.
*   **Stopping Criteria** – The strategy for determining when to kill off poorly peforming runs, and try more combinations faster. We offer several custom scheduling algorithms like [HyperBand](https://arxiv.org/pdf/1603.06560.pdf) and Envelope.
*   **Parameters** – A dictionary containing the hyperparameter names, and discreet values, max and min values or distributions from which to pull their values to sweep over.

*Note: you can also use other search libraries such as [Ray-Tune](https://docs.wandb.com/sweeps/ray-tune) or even [create your own search algorithm](https://docs.wandb.com/sweeps/python-api#run-a-local-controller)*

Here we just use a Python object. Refer to the [sweep configuration documentation](https://docs.wandb.com/sweeps/configuration) if you want to define more parameters.

In [6]:
sweep_config = {
  "method": "grid",   # bayes search
  "metric": {           # We want to minimize validation loss
      "name": "val_epoch_F1",
      "goal": "maximize"
  },
  "parameters": {
        "batch-size": {
            # Choose from pre-defined values
            "values": [16]
        },
        "seq-len": {
            # Choose from pre-defined values
            "values": [700,600,400]
        },
        "lr": {
            "values":[0.0001,0.0002,0.0003]
            # log uniform distribution between exp(min) and exp(max)
            # "distribution": "log_uniform",
            # "min": -9.21,   # exp(-9.21) = 1e-4
            # "max": -4.61    # exp(-4.61) = 1e-2
        },
        "dropout":{
            "values": [0.1,0.15]
        },
        "weight-decay":{
            "values":[0.0003,0.0002,0.00001]
        },
        "em-size":{
            "values":[256,128,64]
        },
        "nhid":{
            "values":[128,64]
        },
        "nlayers":{
            "values":[4,3,2]
        },
        "nhead":{
            "values":[8,4,2]
        }

    }
}

## Running an agent

We now just need to run an agent against the sweep configuration. Sweeps can be run in multiple ways:
* through a command line with `wandb sweep my_sweep_config.yaml`
* directly within a script/notebook with the [Python sweep API](https://docs.wandb.com/sweeps/python-api)

To use the Python sweep API, we define a function that the agent will run, refering to sweep parameters with `wandb.config.my_param`.

In [7]:


def sweep_iteration():
    # set up W&B logger
    wandb.init()    # required to have access to `wandb.config`
    wandb_logger = WandbLogger()

    # setup data
    config_data = {
    'batch-size' :wandb.config["batch-size"], # Batch Size 
    'seq-len' : wandb.config['seq-len'], # Sequence length
    'filter-seq-len' :350 # remove sequence whose size is greater than this len
    }

    dm = MyDataModule(dir_path=dir_name,file_name=fname,config=config_data)


    config_model = {
        'lr' : wandb.config['lr'],
        'dropout' : wandb.config['dropout'],
        'weight-decay': wandb.config['weight-decay'],
        'em-size' :wandb.config['em-size'], # embedding dimension 
        'nhid' : wandb.config['nhid'], # the dimension of the feedforward network model in nn.TransformerEncoder
        'nlayers' :wandb.config['nlayers'], # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
        'nhead' : wandb.config['nhead'], # the number of heads in the multiheadattention models
        'seq-len': config_data['seq-len'], # dont use wandb config 
        'vocab-size':len(dm.vocab.stoi) # the size of vocabulary /also called tokens
    }

    # setup model - note how we refer to sweep parameters with wandb.config
    model = TransformerModel(config=config_model)

    # setup Trainer
    trainer = Trainer(precision=16,gpus=-1, num_nodes=1,  max_epochs=1000, check_val_every_n_epoch=1,deterministic=True, logger=[wandb_logger] ,gradient_clip_val=0.5,enable_pl_optimizer=True,callbacks=[early_stop_callback],progress_bar_refresh_rate=0)
    trainer.fit(model,dm) # traning and validation

    return None

# sweep_id = wandb.sweep(sweep_config, project="Alarm-Transformers-Net") # sweep id based on our configuration
# wandb.agent(sweep_id, function=sweep_iteration)

### Learning Rate Finder

In [8]:
# model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout,seq_len=seq_len)
# trainer = Trainer(precision=16,gpus=1,max_epochs=400,check_val_every_n_epoch=4,deterministic=True, gradient_clip_val=0.5,logger=tb_logger,progress_bar_refresh_rate=50,auto_lr_find=0.002)
# trainer.tune(model,dm) # finding the lr : first way
# 2nd way
# lr_finder = trainer.tuner.lr_find(model)
# print(lr_finder.results)
# fig = lr_finder.plot(suggest=True) # Plot with
# fig.show()
# new_lr = lr_finder.suggestion() # Pick point based on plot, or get suggestion



In [9]:


# config_data = {
#     'batch-size' : 64, # Batch Size 
#     'seq-len' : 900, # Sequence length
#     'filter-seq-len' :350 # remove sequence whose size is greater than this len
# }

# dm = MyDataModule(dir_path=dir_name,file_name=fname,config=config_data)

# config_model = {
#     'lr' : 0.0001,
#     'dropout' : 0.0,
#     'weight-decay': 0.0,
#     'em-size' :256, # embedding dimension 
#     'nhid' : 256, # the dimension of the feedforward network model in nn.TransformerEncoder
#     'nlayers' :2, # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
#     'nhead' : 2, # the number of heads in the multiheadattention models
#     'seq-len': config_data['seq-len'],
#     'vocab-size':len(dm.vocab.stoi) # the size of vocabulary /also called tokens
# }

# print(f"> Vocab Size (Number of Unique Alarms): {config_model['vocab-size']}")
# model = TransformerModel(config=config_model)
# print(f"> Customised lr = {model.lr}")

# Testing

In [10]:
# trainer.test(datamodule=dm) # testing

In [11]:
# accelerator='dp'
# progress_bar_refresh_rate=0 # set to zero to disable it

In [12]:
# wandb.finish() # When we want to close our W&B run, we can call wandb.finish() (mainly useful in notebooks, called automatically in scripts).

In [13]:
import shutil
import tempfile
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.utilities.cloud_io import load as pl_load
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler, PopulationBasedTraining
from ray.tune.integration.pytorch_lightning import TuneReportCallback, TuneReportCheckpointCallback


Ray Tune comes with ready-to-use PyTorch Lightning callbacks. To report metrics back to Tune after each validation epoch, we will use the TuneReportCallback:

We are also able to specify the number of epochs to train each model, and the number of GPUs we want to use for training. We also create a TensorBoard logger that writes logfiles directly into Tune’s root trial directory - if we didn’t do that PyTorch Lightning would create subdirectories, and each trial would thus be shown twice in TensorBoard, one time for Tune’s logs, and another time for PyTorch Lightning’s logs.

In [14]:
def train_alarmnet_tune(config,num_epochs, dir_name, fname):
    early_stop_callback = EarlyStopping(
        monitor='val_epoch_loss',
        min_delta=0.00,
        patience=6,
        verbose=True,
        mode='min'
    )

    
    tuner_callback = TuneReportCallback({
    "val_epoch_loss": "val_epoch_loss",
    "val_epoch_F1": "val_epoch_F1"
    }, on="validation_end")

    # setup data
    config_data = {
    'batch-size' :config["batch-size"], # Batch Size 
    'seq-len' : config['seq-len'], # Sequence length
    'filter-seq-len':350 # remove sequence whose size is greater than this len
    }

    dm = MyDataModule(dir_path=dir_name,file_name=fname,config=config_data)


    config_model = {
        'lr' : config['lr'],
        'dropout' : config['dropout'],
        'weight-decay': config['weight-decay'],
        'em-size' :config['em-size'], # embedding dimension 
        'nhid' : config['nhid'], # the dimension of the feedforward network model in nn.TransformerEncoder
        'nlayers' :config['nlayers'], # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
        'nhead' : config['nhead'], # the number of heads in the multiheadattention models
        'seq-len': config_data['seq-len'], # dont use wandb config 
        'vocab-size':len(dm.vocab.stoi) # the size of vocabulary /also called tokens
    }

    # setup model - note how we refer to sweep parameters with wandb.config
    model = TransformerModel(config=config_model)

    # setup Trainer
    # logger=[wandb_logger]
    tb_logger =  TensorBoardLogger(save_dir=tune.get_trial_dir(), name="", version=".")
    trainer = Trainer(precision=16,gpus=-1, num_nodes=1,  max_epochs=100, check_val_every_n_epoch=1,deterministic=True, logger = [tb_logger],  gradient_clip_val=0.5,enable_pl_optimizer=True,callbacks=[early_stop_callback,tuner_callback],progress_bar_refresh_rate=0)
    trainer.fit(model,dm) # traning and validation
    return None
    

In this example, we use an Asynchronous Hyperband scheduler. This scheduler decides at each iteration which trials are likely to perform badly, and stops these trials. This way we don’t waste any resources on bad hyperparameter configurations.

In [15]:
def tune_alarmnet_asha(num_samples, num_epochs, gpus_per_trial=0):
    dir_name = "/home/waris/Github/predict-future-alarms/.data/"
    fname = 'seqs.tokens'
    config_tune = {
        "batch-size":  tune.choice([16]), 
        "seq-len": tune.grid_search([700]),        
        "lr": tune.grid_search([0.0001,0.0002]),
        "dropout": tune.grid_search ( [0.1,0.15]),
        "weight-decay": tune.grid_search([0.0003,0.0002]),
        "em-size":  tune.grid_search([256,512]),
        "nhid": tune.grid_search([266,128,64]),
        "nlayers": tune.grid_search([4,3,2]),
        "nhead": tune.grid_search([8,4])
        }

    scheduler = ASHAScheduler(max_t=num_epochs,grace_period=1,reduction_factor=2)

    # reporter = CLIReporter(
    #     parameter_columns=["seq-len","lr","dropout","weight-decay","em-size","nhid","nlayers","nhead"],
    #     metric_columns=["val_epoch_loss","val_epoch_F1","training_iteration"]
    # )

    analysis = tune.run(
        tune.with_parameters(train_alarmnet_tune,num_epochs=num_epochs,dir_name=dir_name,fname=fname),
        resources_per_trial={"cpu": 4,"gpu":1},
        metric="val_epoch_F1",
        mode="max",
        config=config_tune,
        num_samples=num_samples,
        scheduler=scheduler,
        # progress_reporter=reporter,
        name="tune_alarmnet_asha")

    print("Best hyperparameters found were: ", analysis.best_config)

    shutil.rmtree(dir_name+"ray-tune")

Ray Tune will now proceed to sample 10 different parameter combinations randomly, train them, and compare their performance afterwards.

In [16]:
tune_alarmnet_asha(num_samples=10, num_epochs=100)


2021-01-22 05:57:25,195	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8266
2021-01-22 05:57:25,845	WARNING function_runner.py:539 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,batch-size,dropout,em-size,lr,nhead,nhid,nlayers,seq-len,weight-decay
_inner_8e95b_00000,RUNNING,,16,0.1,256,0.0001,8,128,3,700,0.0003


0lines [00:00, ?lines/s]
4082lines [00:00, 40802.92lines/s]
6106lines [00:00, 31267.11lines/s]
7967lines [00:00, 25968.36lines/s]
9661lines [00:00, 22387.28lines/s]
11370lines [00:00, 20481.01lines/s]
13014lines [00:00, 16809.44lines/s]
14521lines [00:00, 15782.06lines/s]
15990lines [00:00, 15256.60lines/s]
17443lines [00:00, 14605.68lines/s]
18859lines [00:01, 13773.59lines/s]
20215lines [00:01, 13543.11lines/s]
21555lines [00:01, 13190.76lines/s]
22874lines [00:01, 13189.87lines/s]
24226lines [00:01, 13285.82lines/s]
25551lines [00:01, 12942.75lines/s]
26889lines [00:01, 13069.79lines/s]
28207lines [00:01, 13102.53lines/s]
29518lines [00:01, 12998.55lines/s]
30825lines [00:02, 15381.18lines/s]
(pid=89603) total seqs= 27975
(pid=89603) ['A17 A75 A17 A57 A17 A99 A98 A56\n', 'A245 A246 A50 A243\n', 'A243 A9 A559 A1025\n', 'A50 A59 A60 A64 A392 A726 A9 A725 A726 A725 A243 A725\n']
(pid=89603) GPU available: True, used: True
(pid=89603) TPU available: None, using: 0 TPU cores
(pid=89603) 

# Using Population Based Training to find the best parameters (2nd way)

In [17]:
# def train_alarmnet_tune_checkpoint(config=None,num_epochs=None,checkpoint_dir=None,dir_name=None, fname=None):
#     early_stop_callback = EarlyStopping(
#         monitor='val_epoch_loss',
#         min_delta=0.00,
#         patience=6,
#         verbose=True,
#         mode='min'
#     )

    
#     # tuner_callback = TuneReportCallback({
#     # "val_epoch_loss": "val_epoch_loss",
#     # "val_epoch_F1": "val_epoch_F1"
#     # }, on="validation_end")

#     tuner_callback = TuneReportCheckpointCallback(
#                 metrics={
#                     "val_epoch_loss": "val_epoch_loss",
#                     "val_epoch_F1": "val_epoch_F1"
#                 },
#                 filename="checkpoint",
#                 on="validation_end")


#     # setup data
#     config_data = {
#     'batch-size' :config["batch-size"], # Batch Size 
#     'seq-len' : config['seq-len'], # Sequence length
#     'filter-seq-len':350 # remove sequence whose size is greater than this len
#     }

#     dm = MyDataModule(dir_path=dir_name,file_name=fname,config=config_data)


#     config_model = {
#         'lr' : config['lr'],
#         'dropout' : config['dropout'],
#         'weight-decay': config['weight-decay'],
#         'em-size' :config['em-size'], # embedding dimension 
#         'nhid' : config['nhid'], # the dimension of the feedforward network model in nn.TransformerEncoder
#         'nlayers' :config['nlayers'], # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
#         'nhead' : config['nhead'], # the number of heads in the multiheadattention models
#         'seq-len': config_data['seq-len'], # dont use wandb config 
#         'vocab-size':len(dm.vocab.stoi) # the size of vocabulary /also called tokens
#     }

#     model = None

#     if checkpoint_dir:
#         print("-----------------------------------------------------------------------------------------")
#         # Currently, this leads to errors:
#         # model = LightningMNISTClassifier.load_from_checkpoint(
#         #     os.path.join(checkpoint, "checkpoint"))
#         # Workaround:
#         ckpt = pl_load(os.path.join(checkpoint_dir, "checkpoint"),map_location=lambda storage, loc: storage)
#         model = TransformerModel._load_model_state(ckpt, config=config_model)
#         trainer.current_epoch = ckpt["epoch"]
#     else:
#         model = TransformerModel(config=config_model)

#     # setup model - note how we refer to sweep parameters with wandb.config
#     # model = TransformerModel(config=config_model)

#     # setup Trainer
#     # logger=[wandb_logger]
#     tb_logger =  TensorBoardLogger(save_dir=tune.get_trial_dir(), name="", version=".")
#     trainer = Trainer(precision=16,gpus=-1, num_nodes=1,  max_epochs=100, check_val_every_n_epoch=1,deterministic=True, logger = [tb_logger],  gradient_clip_val=0.5,enable_pl_optimizer=True,callbacks=[early_stop_callback,tuner_callback],progress_bar_refresh_rate=0)


#     trainer.fit(model,dm) # traning and validation
#     return None
    

In [18]:
# def tune_alarmnet_pbt(num_samples, num_epochs, gpus_per_trial=0):
#     dir_name = "/home/waris/Github/predict-future-alarms/.data/"
#     fname = 'seqs.tokens'
#     config_tune = {
#         "batch-size": 16, 
#         "seq-len": tune.grid_search([700,600,400]),        
#         "lr": 0.0001,
#         "dropout": tune.grid_search ( [0.1,0.15]),
#         "weight-decay": tune.grid_search([0.0003,0.0002]),
#         "em-size":  tune.grid_search([256,512]),
#         "nhid": tune.grid_search([128,64]),
#         "nlayers": tune.grid_search([3,2]),
#         "nhead": tune.grid_search([8,4])
#         }

#     # scheduler = ASHAScheduler(max_t=num_epochs,grace_period=1,reduction_factor=2)

#     scheduler = PopulationBasedTraining(
#         perturbation_interval=4,
#         hyperparam_mutations={
#             "lr": [0.0001,0.0002],
#             "batch-size": [8,16]
#         })

#     # reporter = CLIReporter(
#     #     parameter_columns=["seq-len","lr","dropout","weight-decay","em-size","nhid","nlayers","nhead"],
#     #     metric_columns=["val_epoch_loss","val_epoch_F1","training_iteration"]
#     # )

#     analysis = tune.run(
#         tune.with_parameters(train_alarmnet_tune_checkpoint,num_epochs=num_epochs,dir_name=dir_name,fname=fname),
#         resources_per_trial={"cpu": 4,"gpu":1},
#         metric="val_epoch_F1",
#         mode="max",
#         config=config_tune,
#         num_samples=num_samples,
#         scheduler=scheduler,
#         # progress_reporter=reporter,
#         name="tune_alarmnet_pbt")

#     print("Best hyperparameters found were: ", analysis.best_config)

#     shutil.rmtree(dir_name+"ray-tune")

In [ ]:
# tune_alarmnet_pbt(num_samples=2,num_epochs=2)